# Восстановление золота из руды

## Задача

Заказчик разрабатывает решение для эффективного восстановления золота из золотосодержащей руды. Требуется построить две модели, которые будут предсказывать эффективность восстановления отталкиваясь от параметров процесса. Модели помогут оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками. В нашем распоряжении исторические данные с параметрами процессов и их результатами.

Процесс восстановления золота делится на три этапа: флотация (rougher) и первый и второй этапы очистки (primary_cleaner и secondary_cleaner). Одна модель должна предсказывать эффективность восстановления после флотации, а другая - после всего процесса. Требуется достичь наименьшего итогового sMAPE (Symmetric Mean Absolute Percentage Error), где итоговое sMAPE это 0.25 * sMAPE модели флотации + 0.75 * sMAPE модели всего процесса.

## Подготовка данных

### Импорт библиотек и настройка

In [1]:
import math
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config
from sklearn.metrics import make_scorer
from sklearn.base import BaseEstimator

set_config(display='diagram')
pd.options.display.max_columns = None

### Импорт данных и осмотр

Даны три таблицы. Одна таблица содержит в себе полные данные. Две другие дублируют данные из полной таблицы. Эти две таблицы - тренировочная и тестовая выборки, которые за нас уже определили. Попробуем открыть все таблицы.

In [2]:
df_full = pd.read_csv('./datasets/gold_recovery_full_new.csv')
df_train = pd.read_csv('./datasets/gold_recovery_train_new.csv')
df_test = pd.read_csv('./datasets/gold_recovery_test_new.csv')

display(df_full.info())
display(df_train.info())
display(df_test.info())

display(df_full.head())
display(df_train.head())
display(df_test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19439 entries, 0 to 19438
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                19439 non-null  object 
 1   final.output.concentrate_ag                         19438 non-null  float64
 2   final.output.concentrate_pb                         19438 non-null  float64
 3   final.output.concentrate_sol                        19228 non-null  float64
 4   final.output.concentrate_au                         19439 non-null  float64
 5   final.output.recovery                               19439 non-null  float64
 6   final.output.tail_ag                                19438 non-null  float64
 7   final.output.tail_pb                                19338 non-null  float64
 8   final.output.tail_sol                               19433 non-null  float64


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14149 entries, 0 to 14148
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                14149 non-null  object 
 1   final.output.concentrate_ag                         14148 non-null  float64
 2   final.output.concentrate_pb                         14148 non-null  float64
 3   final.output.concentrate_sol                        13938 non-null  float64
 4   final.output.concentrate_au                         14149 non-null  float64
 5   final.output.recovery                               14149 non-null  float64
 6   final.output.tail_ag                                14149 non-null  float64
 7   final.output.tail_pb                                14049 non-null  float64
 8   final.output.tail_sol                               14144 non-null  float64


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5290 entries, 0 to 5289
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5290 non-null   object 
 1   primary_cleaner.input.sulfate               5286 non-null   float64
 2   primary_cleaner.input.depressant            5285 non-null   float64
 3   primary_cleaner.input.feed_size             5290 non-null   float64
 4   primary_cleaner.input.xanthate              5286 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5290 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5290 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5290 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5290 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5290 non-null   float64
 10  primary_clea

None

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,10.128295,7.25,0.988759,8.547551,10.389648,19.529297,34.174427,14.936526,2.534912,7.476074,2.106679,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,1554.367432,-493.428131,41885.707031,3481.779053,3520.337158,2.838687,6.100378,2.284912,523.546326,55.486599,36.808594,6.486150,11.986616,6.007990,11.836743,6.005818,11.500771,7.101074,28.029297,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,999.706909,-404.066986,1603.011353,-434.715027,1602.375000,-442.204468,1598.937256,-451.294128,1404.472046,-455.462982,1416.354980,-451.939636,14.500184,4.694824,8.764648,2.606185,25.853109,-498.526489,23.893660,-501.406281,23.961798,-495.262817,21.940409,-499.340973,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,10.296251,7.25,1.002663,8.558743,10.

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,10.128295,7.25,0.988759,8.547551,10.389648,19.529297,34.174427,14.936526,2.534912,7.476074,2.106679,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,1554.367432,-493.428131,41885.707031,3481.779053,3520.337158,2.838687,6.100378,2.284912,523.546326,55.486599,36.808594,6.486150,11.986616,6.007990,11.836743,6.005818,11.500771,7.101074,28.029297,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,999.706909,-404.066986,1603.011353,-434.715027,1602.375000,-442.204468,1598.937256,-451.294128,1404.472046,-455.462982,1416.354980,-451.939636,14.500184,4.694824,8.764648,2.606185,25.853109,-498.526489,23.893660,-501.406281,23.961798,-495.262817,21.940409,-499.340973,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,10.296251,7.25,1.002663,8.558743,10.

,date,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-09-01 00:59:59,210.800909,14.993118,8.080000,1.005021,1398.981301,-500.225577,1399.144926,-499.919735,1400.102998,-500.704369,1399.001695,-499.485144,13.128569,5.636710,489.793655,62.710452,42.022440,12.084103,16.922877,6.153479,16.867518,6.150709,1001.849317,-350.301265,1249.738612,-399.107531,1249.750520,-399.396730,1198.287855,-399.488594,999.472250,-399.530962,949.566189,-398.180807,24.938200,-500.490963,14.947607,-500.013837,20.018166,-450.395995,13.987058,-449.831513,12.023554,-497.795834,8.016656,-501.289139,7.946562,-432.317850,4.872511,-500.037437,26.705889,-499.709414
1,2016-09-01 01:59:59,215.392455,14.987471,8.080000,0.990469,1398.777912,-500.057435,1398.055362,-499.778182,1396.151033,-499.240168,1399.508091,-500.416324,13.036280,5.526065,490.104534,61.961163,41.187903,11.918977,17.002532,5.999043,16.996063,6.002315,998.690785,-350.425704,1248.395457,-399.946025,1249.514451,-399.630810,1200.506459,-399.942434,1000.002114,-399.491528,950.200009,-405.787514,24.923466,-499.813963,14.930873,-500.764452,19.988503,-450.114024,14.092981,-450.059163,12.058140,-498.695773,8.130979,-499.634209,7.958270,-525.839648,4.878850,-500.162375,25.019940,-499.819438
2,2016-09-01 02:59:59,215.259946,12.884934,7.786667,0.996043,1398.493666,-500.868360,1398.860436,-499.764529,1398.075709,-502.151509,1399.497088,-499.715479,13.138412,5.426561,489.617969,66.903807,42.546294,12.091332,16.992520,5.850632,16.982096,5.853594,998.516720,-349.783538,1247.440748,-400.264330,1248.207204,-401.074405,1199.770026,-400.790582,999.925519,-399.236999,950.320481,-400.864340,24.908732,-500.304035,14.996511,-500.993821,20.039689,-450.263164,14.077881,-449.660997,11.962366,-498.767484,8.096893,-500.827423,8.071056,-500.801673,4.905125,-499.828510,24.994862,-500.622559
3,2016-09-01 03:59:59,215.336236,12.006805,7.640000,0.863514,1399.618111,-498.863574,1397.440120,-499.211024,1400.129303,-498.355873,1401.065165,-501.038738,12.400507,5.114758,476.618130,59.865919,41.060195,12.181597,16.532144,5.801935,16.515277,5.803657,1000.276604,-350.172597,1251.322675,-398.655289,1250.493702,-399.746728,1199.398691,-397.50

В некоторых столбцах есть пропуски. Категориальных признаков нет (не считая `date`, который не считается за признак).

### Проверка на противоречия между таблицами

Надо убедиться, что сумма количества записей в тренировочной и тестовой выборках равна количеству записей в полной таблице.

In [3]:
df_full.shape[0] == df_train.shape[0] + df_test.shape[0]

True

Всё верно.

Проверяем все три таблицы на полные дубликаты.

In [4]:
print('Full:', df_full.duplicated().sum())
print('Train:', df_train.duplicated().sum())
print('Test:', df_test.duplicated().sum())

Full: 0
Train: 0
Test: 0


Полных дубликатов нет.

У записей нет идентификаторов. Но есть столбец со временем эксперимента. Проверим, уникальные ли они.

In [5]:
print('Full:', df_full['date'].duplicated().sum())
print('Train:', df_train['date'].duplicated().sum())
print('Test:', df_test['date'].duplicated().sum())

Full: 0
Train: 0
Test: 0


В каждой таблице значения времени уникальны. Ими можно пользоваться вместо идентификаторов.

Таблица тренировочной выборки должна дублировать данные из полной таблицы. Все записи в тренировочной выборке должны содержатся и в полной таблице. Проверим это. Сначала проверим только по времени.

In [6]:
df_full.merge(df_train, on='date').shape[0] == df_train.shape[0]

True

Всё правильно. Теперь проверяем, что записи из тренировочной выборки дублируют данные из полной таблицы и по содержанию тоже. Объединим обе таблицы во временную. Во временной таблице должно быть столько же дубликатов, сколько записей в тренировочной выборке.

In [7]:
df_temp = pd.concat([df_full, df_train])
df_temp.duplicated().sum() == df_train.shape[0]

False

Количество дубликатов не совпадает с размером тренировочной выборки. Проверим, что размерность таблицы равна ожидаемой.

In [8]:
df_temp.shape == (df_full.shape[0] + df_train.shape[0], df_full.shape[1])

True

Размерность таблицы правильная. Проверим количество дубликатов в столбце `date`.

In [9]:
df_temp['date'].duplicated().sum() == df_train.shape[0]

True

Количество дубликатов в столбце `date` правильное. Проблема именно в содержимом. Выведем количество несовпадающих строк по содержимому из тех, которые совпадают по дате.

In [10]:
df_train.shape[0] - df_temp.duplicated().sum()

3092

3092 проблемных строки.

Проверим, в чём именно разница.

In [11]:
df_temp = df_temp[df_temp['date'].duplicated(keep=False)]
df_temp = df_temp[~df_temp.duplicated(keep=False)].sort_values(by='date').reset_index(drop=True)

for i in range(0, 6, 2):
    for column in df_temp.columns:
        if df_temp.loc[i, column] != df_temp.loc[i + 1, column]:
            print(f'{i} {column}: {(df_temp.loc[i, column], df_temp.loc[i + 1, column])}')

0 rougher.input.feed_size: (57.51064892280578, 57.510648922805785)
2 secondary_cleaner.state.floatbank5_b_level: (-499.36475207718087, -499.3647520771809)
4 secondary_cleaner.state.floatbank2_a_air: (25.93802616545193, 25.938026165451927)


В трёх из несовпадающих пар несовпадения небольшие и встречаются только в одном из столбцов в каждой. Но столбцы разные. На сколько нам известно, самые актуальные данные в полной таблице. Просто, не было известно, что это касается тренировочных данных тоже. Тренировочную выборку нужно взять из полной таблицы используя значения из столбца `date` текущей тренировочной выборки. Тестовую выборку тоже нужно обновить. Там вовсе не хватает столбцов.

Но сначала проверим тестовые данные. Таблица тестовой выборки должна дублировать данные из полной таблицы. Все записи в тестовой выборке должны содержатся и в полной таблице. Проверим только по времени. Мы уже знаем, что содержание придётся обновить.

In [12]:
df_full.merge(df_test, on='date').shape[0] == df_test.shape[0]

True

Всё правильно. На всякий случай проведём избыточную проверку на то, что тренировочная и тестовая выборки действительно не пересекаются.

In [13]:
df_train.merge(df_test, on='date').shape[0]

0

Всё в порядке. Обновим тренировочную и тестовую выборки на предобработке.

In [14]:
df_temp = None

### Проверка имеющихся вычислений

В таблице уже присутствуют некоторые вычисления, которые можно вычислить на основании данных из этой же таблицы. Неизвестно, можно ли им доверять, поэтому будем проводить собственные вычисления по надобности. А имеющиеся столбцы с вычислениями удалим.

Проверим адекватность вычислений на примере столбца `rougher.output.recovery`. Этот столбец и столбцы, которые нужны для вычисления значения этого столбца, не имеют пропусков. Сначала выясним, в долях единицы или в процентах посчитан `recovery`.

In [15]:
df_full['rougher.output.recovery'].head()

0    87.107763
1    86.843261
2    86.842308
3    87.226430
4    86.688794
Name: rougher.output.recovery, dtype: float64

Вычисления посчитаны в процентах. Теперь вычислим собственный `recovery`.

In [16]:
def calculate_recovery(concentrate_before, concentrate_after, tail):
    return ((concentrate_after * (concentrate_before - tail)) /
            (concentrate_before * (concentrate_after - tail))) * 100

def make_recovery_series(df, stage):
    return df.apply(
        lambda row: calculate_recovery(
            row['rougher.input.feed_au'],
            row[stage + '.output.concentrate_au'],
            row[stage + '.output.tail_au']
        ),
        axis=1
    )

df_full['rougher.output.recovery_our'] = make_recovery_series(df_full, 'rougher')

df_full[['rougher.output.recovery', 'rougher.output.recovery_our']].head()

,rougher.output.recovery,rougher.output.recovery_our
0,87.107763,87.107763
1,86.843261,86.843261
2,86.842308,86.842308
3,87.226430,87.226430
4,86.688794,86.688794


Пока что результаты выглядят одинаковыми. Вычислим MAE.

In [17]:
mean_absolute_error(df_full['rougher.output.recovery_our'], df_full['rougher.output.recovery'])

9.874045668302637e-15

MAE очень маленькая. Возможно, MAE больше в тренировочной выборке. Проверим это. Там тоже нет пропусков в актуальных для задачи столбцах.

In [18]:
df_train['rougher.output.recovery_our'] = df_train.apply(
    lambda row: calculate_recovery(
        row['rougher.input.feed_au'],
        row['rougher.output.concentrate_au'],
        row['rougher.output.tail_au']
    ),
    axis=1
)

mean_absolute_error(df_train['rougher.output.recovery_our'], df_train['rougher.output.recovery'])

9.73512347450521e-15

MAE тоже очень маленькая. Но она не нулевая в обоих случаях. Будем полагаться на собственные вычисления.

### Столбцы, которых нет в старой тестовой выборке

Ещё раз посмотрим на столбцы тестовой выборки.

In [19]:
for column in df_test.columns:
    print(column)

date
primary_cleaner.input.sulfate
primary_cleaner.input.depressant
primary_cleaner.input.feed_size
primary_cleaner.input.xanthate
primary_cleaner.state.floatbank8_a_air
primary_cleaner.state.floatbank8_a_level
primary_cleaner.state.floatbank8_b_air
primary_cleaner.state.floatbank8_b_level
primary_cleaner.state.floatbank8_c_air
primary_cleaner.state.floatbank8_c_level
primary_cleaner.state.floatbank8_d_air
primary_cleaner.state.floatbank8_d_level
rougher.input.feed_ag
rougher.input.feed_pb
rougher.input.feed_rate
rougher.input.feed_size
rougher.input.feed_sol
rougher.input.feed_au
rougher.input.floatbank10_sulfate
rougher.input.floatbank10_xanthate
rougher.input.floatbank11_sulfate
rougher.input.floatbank11_xanthate
rougher.state.floatbank10_a_air
rougher.state.floatbank10_a_level
rougher.state.floatbank10_b_air
rougher.state.floatbank10_b_level
rougher.state.floatbank10_c_air
rougher.state.floatbank10_c_level
rougher.state.floatbank10_d_air
rougher.state.floatbank10_d_level
rougher.st

Вся информация по этим столбцам известна до начала эксперимента. Теперь посмотрим на столбцы, которых не хватает в тестовой выборке.

In [20]:
for column in df_full.columns:
    if column not in df_test.columns:
        print(column)

final.output.concentrate_ag
final.output.concentrate_pb
final.output.concentrate_sol
final.output.concentrate_au
final.output.recovery
final.output.tail_ag
final.output.tail_pb
final.output.tail_sol
final.output.tail_au
primary_cleaner.output.concentrate_ag
primary_cleaner.output.concentrate_pb
primary_cleaner.output.concentrate_sol
primary_cleaner.output.concentrate_au
primary_cleaner.output.tail_ag
primary_cleaner.output.tail_pb
primary_cleaner.output.tail_sol
primary_cleaner.output.tail_au
rougher.calculation.sulfate_to_au_concentrate
rougher.calculation.floatbank10_sulfate_to_au_feed
rougher.calculation.floatbank11_sulfate_to_au_feed
rougher.calculation.au_pb_ratio
rougher.output.concentrate_ag
rougher.output.concentrate_pb
rougher.output.concentrate_sol
rougher.output.concentrate_au
rougher.output.recovery
rougher.output.tail_ag
rougher.output.tail_pb
rougher.output.tail_sol
rougher.output.tail_au
secondary_cleaner.output.tail_ag
secondary_cleaner.output.tail_pb
secondary_cleaner.

Вся информация по этим столбцам становится известна лишь после начала эксперимента. Тестовая выборка содержит все столбцы, которые относятся к информации, которая должна быть известна до начала эксперимента, и никаких больше.

Сомнения вызывает только столбец `primary_cleaner.input.feed_size`. Можно предположить, что это размер продукта, который идёт на вход первого этапа очистки. Возможно, он, на самом деле, должен зависеть от того, что выходит из этапа флотации. Уточнить возможности нет. Будем относиться к этому как к ещё одному параметру для машинного обучения.

### Предобработка

Неактуальные данные в тренировочной и тестовой выборке мы заменим на данные из полной таблицы. Но сначала полную таблицу надо обработать.

Посчитаем пропуски по столбцам в полной таблице.

In [21]:
print(df_full.isna().sum().sort_values(ascending=False).to_string())

secondary_cleaner.output.tail_sol                     1748
rougher.input.floatbank11_xanthate                     453
rougher.state.floatbank10_e_air                        436
primary_cleaner.output.concentrate_sol                 370
secondary_cleaner.state.floatbank2_a_air               220
final.output.concentrate_sol                           211
rougher.input.feed_size                                145
primary_cleaner.output.concentrate_pb                  116
primary_cleaner.input.xanthate                         104
final.output.tail_pb                                   101
rougher.input.feed_pb                                  100
rougher.input.feed_sol                                  99
primary_cleaner.output.tail_sol                         62
rougher.input.floatbank11_sulfate                       44
primary_cleaner.input.depressant                        37
rougher.input.floatbank10_sulfate                       34
primary_cleaner.input.sulfate                           

Пропуски есть как в нужных столбцах для обучения, так и в лишних. Известно, что соседние по времени параметры часто похожи. Но, возможно, будет проще просто удалить строки с пропусками.

Процесс обработки делится на три этапа: флотация, первый этап очистки, второй этап очистки. Нам нужны все столбцы, которые относятся к сырью, которое попадает на каждый этап, параметры каждого этапа, вывод концентрата каждого этапа, кроме концентратов, которые не касаются золота для последнего этапа, и нам нужны все отвальные хвосты золота. Нам не нужны столбцы, которые касаются вычислений, столбцы с не золотыми отвальными хвостами, и столбцы с не золотыми концентратами финального этапа.

Разделим все столбцы на группы. Выводы каждого этапа (кроме последнего) подаются на вход одного из следующих, но сейчас мы не будем учитывать столбцы дважды.

In [22]:
column_arr = {
    'rougher_X': [],
    'rougher_y': ['rougher.output.tail_au'],
    'primary_cleaner_X': [],
    'primary_cleaner_y': ['primary_cleaner.output.tail_au'],
    'secondary_cleaner_X': [],
    'secondary_cleaner_y': [
        'final.output.concentrate_au',
        'final.output.tail_au',
        'secondary_cleaner.output.tail_au'
    ],
    'other': []
}

useful_columns = []

for column in df_full.columns:
    if column.startswith('rougher.input') or column.startswith('rougher.state'):
        column_arr['rougher_X'].append(column)
    elif column.startswith('rougher.output.concentrate'):
        column_arr['rougher_y'].append(column)
    elif column.startswith('primary_cleaner.input') or column.startswith('primary_cleaner.state'):
        column_arr['primary_cleaner_X'].append(column)
    elif column.startswith('primary_cleaner.output.concentrate'):
        column_arr['primary_cleaner_y'].append(column)
    elif column.startswith('secondary_cleaner.state'):
        column_arr['secondary_cleaner_X'].append(column)
    elif column not in column_arr['secondary_cleaner_y'] and\
         column not in column_arr['primary_cleaner_y'] and\
         column not in column_arr['rougher_y']:
        column_arr['other'].append(column)

sum = 0

for key, columns in column_arr.items():
    sum += len(columns)
    print(f'{key}; Length: {len(columns)}')
    
    for column in columns:
        print('   ', column)
        
        if key != 'other':
            useful_columns.append(column)
    
    print()

print('Total columns:', sum)
print('Total useful columns:', len(useful_columns))

rougher_X; Length: 22
    rougher.input.feed_ag
    rougher.input.feed_pb
    rougher.input.feed_rate
    rougher.input.feed_size
    rougher.input.feed_sol
    rougher.input.feed_au
    rougher.input.floatbank10_sulfate
    rougher.input.floatbank10_xanthate
    rougher.input.floatbank11_sulfate
    rougher.input.floatbank11_xanthate
    rougher.state.floatbank10_a_air
    rougher.state.floatbank10_a_level
    rougher.state.floatbank10_b_air
    rougher.state.floatbank10_b_level
    rougher.state.floatbank10_c_air
    rougher.state.floatbank10_c_level
    rougher.state.floatbank10_d_air
    rougher.state.floatbank10_d_level
    rougher.state.floatbank10_e_air
    rougher.state.floatbank10_e_level
    rougher.state.floatbank10_f_air
    rougher.state.floatbank10_f_level

rougher_y; Length: 5
    rougher.output.tail_au
    rougher.output.concentrate_ag
    rougher.output.concentrate_pb
    rougher.output.concentrate_sol
    rougher.output.concentrate_au

primary_cleaner_X; Length: 12
  

Теперь посчитаем количество строк с пропусками в полезных столбцах и их долю от всех строк.

In [23]:
na_rows = df_full[useful_columns].shape[0] - df_full[useful_columns].dropna().shape[0]

print(na_rows)
print(na_rows/df_full.shape[0])

1796
0.09239158392921447


9.24% процента строк имеют пропуски. Придётся их заполнить. Известно, что соседние по времени параметры часто похожи. Отсортируем таблицу по времени и к каждой строке применим функцию. Если в строке в полезном столбце встречается пропуск, то, по возможности, заполняем его значением из тренировочной или тестовой выборки, а иначе заполняем средним значением из двух ближайших соседних строк в большую и меньшую сторону, где нет пропусков в соответствующих столбцах.

In [24]:
# В такой реализации возможны race conditions, но понадеемся, что это не важно.

def fill_na_func(row):
    for column in row.index:
        if column == 'date':
            continue
        
        if math.isnan(row[column]):
            df = df_train[df_train['date'] == row['date']]
            
            if len(df) == 0:
                df = df_test[df_test['date'] == row['date']]
                
            df = df.reset_index()
            
            if column in df.columns:
                val = df.iloc[0][column]
            else:
                val = float('NaN')
            
            if not math.isnan(val):
                row[column] = val
            else:
                if row['idx'] != 0:
                    for i in range(row['idx'] - 1, -1, -1):
                        val = df_full.iloc[i][column]
                        
                        if not math.isnan(val):
                            break
                    
                    if row['idx'] != df_full.shape[0] - 1:
                        val2 = float('NaN')
                        
                        for i in range(row['idx'] + 1, df_full.shape[0]):
                            val2 = df_full.iloc[i][column]

                            if not math.isnan(val2):
                                break
                        
                        if not math.isnan(val2) and not math.isnan(val):
                            row[column] = (val + val2) / 2
                        else:
                            if math.isnan(val):
                                row[column] = val2
                            else:
                                row[column] = val
                    else:
                        row[column] = val
                else:
                    row[column] = df_full.iloc[1][column]
    
    return row

df_full = df_full.sort_values(by='date').reset_index(drop=True)

# Функция, которая будет отрабатывать на каждой строке, похоже,
# не имеет доступа к индексу строки. В документации не нашёл ничего.
# Так что придётся продублировать индекс.
df_full['idx'] = df_full.index

df_full = df_full.apply(fill_na_func, axis=1)

na_rows = df_full[useful_columns].shape[0] - df_full[useful_columns].dropna().shape[0]

print(na_rows)

0


Избавились от всех пропусков.

Теперь можно обновить тренировочную и тестовую выборки.

In [25]:
print('Train shape before:', df_train.shape)
print('Test shape before:', df_test.shape)

df_train = pd.concat([df_full, df_train])
df_train = df_train[df_train['date'].duplicated(keep='last')]

df_test = pd.concat([df_full, df_test])
df_test = df_test[df_test['date'].duplicated(keep='last')]

print('Train shape after:', df_train.shape)
print('Test shape after:', df_test.shape)

Train shape before: (14149, 88)
Test shape before: (5290, 53)
Train shape after: (14149, 89)
Test shape after: (5290, 89)


Проверим, что теперь-то данные в тренировочной и тестовой выборках полностью дублируют данные из полной таблицы

In [26]:
print(pd.concat([df_full, df_train]).duplicated().sum() == df_train.shape[0])
print(pd.concat([df_full, df_test]).duplicated().sum() == df_test.shape[0])

True


True


Мы избавились от неактуальных данных. Противоречий между таблицами больше нет, так как все данные теперь только из полной таблицы.

Всю иную предобработку будем проводить в пайплайнах.

## Анализ данных

### Изменения концентрации между этапами

Отобразим столбчатую диаграмму, которая отразит измененения долей различных металлов в сырье/концентрате в зависимости от шага. Для этого, сначала, построим таблицу для функции построения столбчатой диаграммы.

In [27]:
df_concentrate = pd.DataFrame(columns=[
    'Металл',
    'Шаг',
    'Среднее',
])

metals = [('Золото', 'au'), ('Серебро', 'ag'), ('Свинец', 'pb')]

steps = [
    ('До обработки', 'rougher.input.feed_'),
    ('После флотации', 'rougher.output.concentrate_'),
    ('После первичной очистки', 'primary_cleaner.output.concentrate_'),
    ('После второй очистки (финал)', 'final.output.concentrate_')
]

i = 0

for metal in metals:
    for step in steps:
        df_concentrate.loc[i] = [
            metal[0],
            step[0],
            df_full[step[1] + metal[1]].mean()
        ]
        
        i += 1

df_concentrate

,Металл,Шаг,Среднее
0,Золото,До обработки,8.266097
1,Золото,После флотации,19.772000
2,Золото,После первичной очистки,32.119400
3,Золото,После второй очистки (финал),44.076513
4,Серебро,До обработки,8.794927
5,Серебро,После флотации,11.994759
6,Серебро,После первичной очистки,8.442408
7,Серебро,После второй очистки (финал),5.168385
8,Свинец,До обработки,3.585473
9,Свинец,После флотации,7.606272


Построим столбчатую диаграмму.

In [28]:
px.bar(df_concentrate, x='Шаг', y='Среднее', color='Металл', barmode='group')

Видно, что процесс направлен именно на выделение золота. Доля серебра совсем немного увеличилась после флотации и начала падать после. Для свинца немного выросла после флотации и после первичной очистки и осталась такой же после второй очистки. Зато доля золота вырастает на 10 с лишним процентов после каждого этапа.

Несмотря на то, что доля золота вырастает примерно на одинаковое количество процентов, если измерять эффективность по доле удалённых излишков от всех излишков, то эффективность растёт. Построим ещё одну столбчатую диаграмму.

In [29]:
df_temp = df_concentrate.query('Металл == "Золото"').copy()
df_temp['Удалённые излишки'] = pd.Series(0)

for i in range(1, 4):
    df_temp.loc[i, 'Удалённые излишки'] = (
        (df_temp.loc[i]['Среднее'] - df_temp.loc[i - 1]['Среднее']) /
        (100 - df_temp.loc[i - 1]['Среднее'])
    ) * 100
    
px.bar(df_temp.query('Шаг != "До обработки"'), x='Шаг', y='Удалённые излишки')

Эффективность немного, но растёт.

Посмотрим на изменение суммарной концентрации всех веществ, которые фигурируют в таблице. Добавим к нашей таблице соли.

In [30]:
i = df_concentrate.shape[0]

for step in steps:
    df_concentrate.loc[i] = [
        'Соль',
        step[0],
        df_full[step[1] + 'sol'].mean()
    ]

    i += 1


df_concentrate = df_concentrate.rename(columns={'Металл': 'Вещество'})

df_concentrate

,Вещество,Шаг,Среднее
0,Золото,До обработки,8.266097
1,Золото,После флотации,19.772000
2,Золото,После первичной очистки,32.119400
3,Золото,После второй очистки (финал),44.076513
4,Серебро,До обработки,8.794927
5,Серебро,После флотации,11.994759
6,Серебро,После первичной очистки,8.442408
7,Серебро,После второй очистки (финал),5.168385
8,Свинец,До обработки,3.585473
9,Свинец,После флотации,7.606272


Построим ещё одну столбчатую диаграмму.

In [31]:
px.bar(df_concentrate, x='Шаг', y='Среднее', color='Вещество')

Суммарная концентрация всех отслеживаемых веществ то растёт, то падает, но находится в районе примерно между 55% и 70%. Концентрация соли сильно сокращается после первичной очистки.

### Проверка распределения размеров гранул сырья

Надо перепроверить распределения размеров гранул сырья в тренировочной и тестовой выборках. Если распределения сильно отличаются друг от друга, оценка модели будет неправильной. Построим Violin-plot по обеим выборкам.

In [32]:
fig = go.Figure()

fig.add_trace(go.Violin(
    y=df_train['rougher.input.feed_size'],
    name='Training'
))

fig.add_trace(go.Violin(
    y=df_test['rougher.input.feed_size'],
    name='Testing'
))

fig.show()

Если уменьшить масштаб (график интерактивный), то видно, что распределения отличаются, но не очень сильно.

## Модель

### Подготовка

Теперь, когда аналитика закончена, можно дропнуть лишние столбцы.

In [33]:
df_full = df_full[useful_columns]
df_train = df_train[useful_columns]
df_test = df_test[useful_columns]

Заново посчитаем `final.output.recovery` и `rougher.output.recovery`.

In [34]:
df_full['final.output.recovery'] = make_recovery_series(df_full, 'final')
df_train['final.output.recovery'] = make_recovery_series(df_full, 'final')

df_full['rougher.output.recovery'] = make_recovery_series(df_full, 'rougher')
df_train['rougher.output.recovery'] = make_recovery_series(df_full, 'rougher')

Обработаем тестовую выборку. Выделим Recovery в отдельные переменные, а данные, которые не должны быть известны, занулим.

(Далее будет следовать не очень элегантное решение. Удалять столбцы не получится, если не переделывать гору кода. Но, для чистоты эксперимента, можно столбцы занулить, хотя это можно не делать.)

In [35]:
final_recovery = make_recovery_series(df_full, 'final')
rougher_recovery = make_recovery_series(df_full, 'rougher')

df_test_old = pd.read_csv('./datasets/gold_recovery_test_new.csv')

for column in [*df_test.columns, 'final.output.recovery', 'rougher.output.recovery']:
    if column not in df_test_old.columns:
        df_test[column] = pd.Series([0] * len(df_test))

Реализуем функцию sMAPE (Symmetric Mean Absolute Percentage Error). Мы будем ею оценивать модели.

In [36]:
def sMAPE(y, prediction):
    return ((abs(y - prediction) * 2) / (abs(y) + abs(prediction))).sum() / len(y)

sMAPE_scorer = make_scorer(sMAPE, greater_is_better=False)

### Шаг 1 - Флотация

Сделаем пять моделей, которые будут принимать на вход следующие параметры:

In [37]:
column_arr['rougher_X']

['rougher.input.feed_ag',
 'rougher.input.feed_pb',
 'rougher.input.feed_rate',
 'rougher.input.feed_size',
 'rougher.input.feed_sol',
 'rougher.input.feed_au',
 'rougher.input.floatbank10_sulfate',
 'rougher.input.floatbank10_xanthate',
 'rougher.input.floatbank11_sulfate',
 'rougher.input.floatbank11_xanthate',
 'rougher.state.floatbank10_a_air',
 'rougher.state.floatbank10_a_level',
 'rougher.state.floatbank10_b_air',
 'rougher.state.floatbank10_b_level',
 'rougher.state.floatbank10_c_air',
 'rougher.state.floatbank10_c_level',
 'rougher.state.floatbank10_d_air',
 'rougher.state.floatbank10_d_level',
 'rougher.state.floatbank10_e_air',
 'rougher.state.floatbank10_e_level',
 'rougher.state.floatbank10_f_air',
 'rougher.state.floatbank10_f_level']

И которые будут предсказывать следующие параметры:

In [38]:
column_arr['rougher_y']

['rougher.output.tail_au',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.concentrate_au']

In [39]:
pipeline = Pipeline(steps=[
    ('keep', ColumnTransformer(transformers=[
        ('keep', FunctionTransformer(lambda x: x), column_arr['rougher_X'])
    # У нас стоит remainder='drop'.
    # Это позволяет избавиться от утечек целевого признака,
    # несмотря на то, что мы скармливаем моделям почти целый DataFrame.
    ], remainder='drop')),
    ('scale', MinMaxScaler()),
    ('poly', 'passthrough'),
    ('model', 'passthrough')
])

# Все закомментированные модели медленно работают.
params = [
#     {
#         'model': [ElasticNet(random_state=0, tol=0.4)],
#         'model__alpha': [x / 10.0 for x in range(1, 11, 1)],
#         'model__l1_ratio': [x / 10.0 for x in range(0, 11, 1)]
#     },

    {
        'model': [LinearRegression()]
        
#       Полиномиальные фичи тоже медленно работают.
#       'poly': ['passthrough', PolynomialFeatures(2, include_bias=False)]
    },

    {
        'model': [DummyRegressor()],
        'model__strategy': ['mean', 'median'],
        'scale': ['passthrough'],
        'poly': ['passthrough']
    },

#     {
#         'model': [DecisionTreeRegressor(random_state=0)],
#         'scale': ['passthrough'],
#         'poly': ['passthrough'],
#         'model__max_depth': range(1, 11)
#     },

#     {
#         'model': [RandomForestRegressor(random_state=0)],
#         'scale': ['passthrough'],
#         'poly': ['passthrough'],
#         'model__max_depth': range(1, 20),
#         'model__n_estimators': range(20, 101, 10)
#     },
]

model = {}

def make_and_fit_model(column):
    result = GridSearchCV(
        pipeline,
        params,
        cv=5,
        scoring=sMAPE_scorer
    )

    result.fit(
        # Да, прямо вот так, с целевым признаком.
        # У нас стоит remainder='drop',
        # так что на качество модели это отразиться не должно.
        df_train,
        df_train[column]
    )

    print('Model to predict', column)
    print('   ', result.best_params_['model'])
    print(f'    sMAPE: {round(-result.best_score_ * 100, 2)}%')
    
    return result

for column in column_arr['rougher_y']:
    model[column] = make_and_fit_model(column)

Model to predict rougher.output.tail_au
    LinearRegression()
    sMAPE: 29.39%
Model to predict rougher.output.concentrate_ag
    LinearRegression()
    sMAPE: 16.94%
Model to predict rougher.output.concentrate_pb
    LinearRegression()
    sMAPE: 13.37%


Model to predict rougher.output.concentrate_sol
    LinearRegression()
    sMAPE: 15.77%
Model to predict rougher.output.concentrate_au
    LinearRegression()
    sMAPE: 13.06%


### Предсказание Recovery после флотации

Теперь сделаем модель, которая будет считать `rougher.output.recovery` из предсказаний. Точнее, не модель, а просто пайплайн, который будет генерировать предсказания. Сразу проведём тест на тестовых данных.

In [40]:
def make_transformers_steps(columns):
    arr = []
    
    for column in columns:
        arr.append((
            column,
            FunctionTransformer(
                lambda X: pd.DataFrame(
                    model[column].predict(X)
                )
            ),
            df_train.columns
        ))
        
    return arr

def make_recovery_pipeline(phase):
    return Pipeline(steps=[
        ('generator', ColumnTransformer(transformers=[
                ('feed_au', FunctionTransformer(lambda x: x), ['rougher.input.feed_au']),
                make_transformers_steps([phase + '.output.concentrate_au'])[0],
                make_transformers_steps([phase + '.output.tail_au'])[0]
            ], remainder='drop')
        ),
        ('calculate', FunctionTransformer(
            lambda X: make_recovery_series(pd.DataFrame(X, columns=[
                'rougher.input.feed_au',
                phase + '.output.concentrate_au',
                phase + '.output.tail_au'
            ]), phase)
        ))
    ])

pipeline_rougher_recovery = make_recovery_pipeline('rougher')

pipeline_rougher_recovery.fit(df_train)
rougher_recovery_sMAPE = sMAPE(rougher_recovery, pipeline_rougher_recovery.transform(df_test))
rougher_recovery_sMAPE

0.03682889215431738

sMAPE = 3.7%.

### Шаг 2 - Первичная очистка

Теперь сделаем модели, которые принимают на вход эти параметры, которые будут генерировать модели, которые мы уже сделали:

In [41]:
column_arr['rougher_y'][1:]

['rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.concentrate_au']

И эти параметры:

In [42]:
column_arr['primary_cleaner_X']

['primary_cleaner.input.sulfate',
 'primary_cleaner.input.depressant',
 'primary_cleaner.input.feed_size',
 'primary_cleaner.input.xanthate',
 'primary_cleaner.state.floatbank8_a_air',
 'primary_cleaner.state.floatbank8_a_level',
 'primary_cleaner.state.floatbank8_b_air',
 'primary_cleaner.state.floatbank8_b_level',
 'primary_cleaner.state.floatbank8_c_air',
 'primary_cleaner.state.floatbank8_c_level',
 'primary_cleaner.state.floatbank8_d_air',
 'primary_cleaner.state.floatbank8_d_level']

И будут предсказывать эти параметры:

In [43]:
column_arr['primary_cleaner_y']

['primary_cleaner.output.tail_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.concentrate_au']

In [44]:
for val in params:
    val['keep__transformers'] = [[
        ('keep', FunctionTransformer(lambda x: x), column_arr['primary_cleaner_X']),
        *make_transformers_steps(column_arr['rougher_y'][1:])
    ]]

for column in column_arr['primary_cleaner_y']:
    model[column] = make_and_fit_model(column)

Model to predict primary_cleaner.output.tail_au
    LinearRegression()
    sMAPE: 27.2%
Model to predict primary_cleaner.output.concentrate_ag
    LinearRegression()
    sMAPE: 17.35%
Model to predict primary_cleaner.output.concentrate_pb
    DummyRegressor(strategy='median')
    sMAPE: 24.52%
Model to predict primary_cleaner.output.concentrate_sol
    LinearRegression()
    sMAPE: 31.4%
Model to predict primary_cleaner.output.concentrate_au
    DummyRegressor(strategy='median')
    sMAPE: 12.4%


Некоторые линейные регрессии справляются хуже, чем модели, которые всегда предсказывают медиану.

### Шаг 3 - Вторая очистка

Теперь сделаем модели, которые принимают на вход эти параметры, которые будут генерировать модели, которые мы уже сделали:

In [45]:
column_arr['primary_cleaner_y'][1:]

['primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.concentrate_au']

И эти параметры:

In [46]:
column_arr['secondary_cleaner_X']

['secondary_cleaner.state.floatbank2_a_air',
 'secondary_cleaner.state.floatbank2_a_level',
 'secondary_cleaner.state.floatbank2_b_air',
 'secondary_cleaner.state.floatbank2_b_level',
 'secondary_cleaner.state.floatbank3_a_air',
 'secondary_cleaner.state.floatbank3_a_level',
 'secondary_cleaner.state.floatbank3_b_air',
 'secondary_cleaner.state.floatbank3_b_level',
 'secondary_cleaner.state.floatbank4_a_air',
 'secondary_cleaner.state.floatbank4_a_level',
 'secondary_cleaner.state.floatbank4_b_air',
 'secondary_cleaner.state.floatbank4_b_level',
 'secondary_cleaner.state.floatbank5_a_air',
 'secondary_cleaner.state.floatbank5_a_level',
 'secondary_cleaner.state.floatbank5_b_air',
 'secondary_cleaner.state.floatbank5_b_level',
 'secondary_cleaner.state.floatbank6_a_air',
 'secondary_cleaner.state.floatbank6_a_level']

И будут предсказывать `final.output.concentrate_au` и `secondary_cleaner.output.tail_au`.

In [47]:
for val in params:
    val['keep__transformers'] = [[
        ('keep', FunctionTransformer(lambda x: x), column_arr['secondary_cleaner_X']),
        *make_transformers_steps(column_arr['primary_cleaner_y'][1:])
    ]]

model['final.output.concentrate_au'] = make_and_fit_model('final.output.concentrate_au')
model['secondary_cleaner.output.tail_au'] = make_and_fit_model('secondary_cleaner.output.tail_au')

Model to predict final.output.concentrate_au
    DummyRegressor(strategy='median')
    sMAPE: 6.33%
Model to predict secondary_cleaner.output.tail_au
    DummyRegressor(strategy='median')
    sMAPE: 37.16%


Обе модели просто предсказывают медиану. В итоге польза от машинного обучения заключается только золотых хвостах на первых двух шагах.

### Шаг 4 - Финальный золотой хвост

Теперь осталось только как-то вычислить `final.output.tail_au`. Возможно, тут можно было бы обойтись и без машинного обучения, но не до конца ясно, что именно означают значения хвостов, и достаточно ли у нас информации, чтобы вычислить финальный хвост так. Поэтому попробуем предсказать `final.output.tail_au` опираясь на хвосты с предыдущих этапов.

In [48]:
for val in params:
    val['keep__transformers'] = [[
        *make_transformers_steps([
            'rougher.output.tail_au',
            'primary_cleaner.output.tail_au',
            'secondary_cleaner.output.tail_au'
        ])
    ]]

model['final.output.tail_au'] = make_and_fit_model('final.output.tail_au')

Model to predict final.output.tail_au
    DummyRegressor(strategy='median')
    sMAPE: 25.23%


И снова получилась модель, которая всегда предсказывает медиану. Для предсказания Recovery всего процесса будем пользоваться просто медианами без всякого машинного обучения.

### Предсказание Recovery всего процесса

Теперь сделаем пайплайн, который будет считать `final.output.recovery` из предсказаний. Сразу проведём тест на тестовых данных.

In [49]:
pipeline_final_recovery = make_recovery_pipeline('final')
pipeline_final_recovery.fit(df_train)
final_recovery_sMAPE = sMAPE(final_recovery, pipeline_final_recovery.transform(df_test))
final_recovery_sMAPE

0.04671442776582669

sMAPE = 4.67%.

### Итоговое sMAPE

Вычисляем итоговое sMAPE.

In [50]:
final_sMAPE = sMAPE = 0.25 * rougher_recovery_sMAPE + 0.75 * final_recovery_sMAPE
final_sMAPE

0.04424304386294936

sMAPE = 4.42%

### Вывод

Из трёх таблиц, две таблицы это выборки из третьей полной таблицы. Это тренировочная и тестовая выборки. Они должны дублировать данные из полной таблицы, но, они отличаются. Эти выборки было решено обновить данными из полной таблицы. Также в данных пересчитали Recovery, на всякий случай.

Процесс направлен именно на выделение золота. Концентрации других металлов в процессе меняются слабо.

Отличия распределений размеров гранул сырья на обучающей и тестовой выборках в пределах нормы.

У пайплайна, который выдаёт Recovery после флотации sMAPE = 3.7%. Две модели линейной регрессии предсказывают концентрацию золота и золотой хвост, и на основании этих предсказаний происходят вычисления. Пайплайн находится в переменной `pipeline_rougher_recovery`.

У пайплайна, который выдаёт Recovery после всего процесса sMAPE = 4.67%. Машинное обучение не участвует в процессе. Recovery вычисляется из медиан финальной концентрации золота и финального золотого хвоста. Пайплайн находится в переменной `pipeline_final_recovery`.

Итоговое sMAPE = 4.42%.